In [221]:
# import modules
import pandas as pd 
import numpy as np 

from sklearn.preprocessing import (PowerTransformer, 
                                   LabelEncoder)
from sklearn.model_selection import train_test_split

In [222]:
# init global variables
train_path = '/kaggle/input/playground-series-s3e26/train.csv'
test_path = '/kaggle/input/playground-series-s3e26/test.csv'

In [223]:
# read datasets
train_df = pd.read_csv(train_path)
train_df.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C


In [224]:
class GetDummies:
    def __init__(self, name='category', fit=None):
        self.columns = None
        self.fillna_value = None
        self.name = name
        
        if fit is not None: 
            self.fit(fit)
            self.first_fit = True
        else: 
            self.first_fit = False
            
    def fit(self, series):
        self.columns = series.unique()
        self.first_fit = True

    def set_fillna(self, v):
        self.fillna_value = v

    def lst_transform(self, data):
        result = []

        for val in data:
            variants = [0] * len(self.columns)
            not_founded = True

            for i, col in enumerate(self.columns):
                if val == col:
                    variants[i] = 1
                    result.append(variants)
                    not_founded = False
                    break

            if not_founded:
                result.append([self.fillna_value] * len(self.columns))
        return result

    def transform(self, data):
        tr_lst = self.lst_transform(data)

        df_data = {f'{self.name}_{col}': [] for col in self.columns}

        for tr in tr_lst:
            for col, val in zip(self.columns, tr):
                df_data[f'{self.name}_{col}'].append(val)

        return pd.DataFrame(df_data)

    def __call__(self, data):
        if self.first_fit == False: 
            self.fit(data)
            self.first_fit = True
            
        return self.transform(data.to_list())

In [225]:
false_true_cols = ["Sex", "Ascites", "Spiders", "Edema", "Hepatomegaly"]

drug_enc = GetDummies(name = "drug", fit = train_df['Drug'])

def false_true_cols_(df) -> pd.DataFrame: 
    for i in false_true_cols: 
        if i == 'Sex': 
            df[i] = df[i].apply(lambda l: 1 if l == 'F' else 0)
        else:  
            df[i] = df[i].apply(lambda l: 1 if l == 'N' else 0)
    return df 

def preprocess_y(df): 
    t = {'D': 0, 'C': 1, 'CL': 2}
    return df.apply(lambda i: t[i])

def caabstegorical_cols_(df) -> pd.DataFrame: 
    drug_dummies = drug_enc(df['Drug'])
    df = df.drop('Drug', axis=1)
    df = pd.concat([drug_dummies, df], axis=1)
    
    return df

def transform_numeric_cols_(df) -> pd.DataFrame:
    ...
    return df

def preprocess_x(df) -> pd.DataFrame: 
    df = df.copy()
    
    df = df.drop('id', axis=1)
    df = transform_numeric_cols_(df)
    df = false_true_cols_(df)
    df = categorical_cols_(df)
    
    return df

In [229]:
X = preprocess_x(train_df.drop(['Status'], axis=1))
y = preprocess_y(train_df['Status'])

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size = 0.8)

In [230]:
from tensorflow.data import Dataset
import tensorflow.keras.layers as l
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
import tensorflow as tf

In [231]:
def build_dataset(X, y, batch_size = 16, shuffle = True, prefetch = True): 
    dataset = Dataset.from_tensor_slices((
        X, 
        y)
    ).batch(batch_size)
    
    if shuffle: 
        dataset = dataset.shuffle(16)
        
    if prefetch: 
        dataset = dataset.prefetch(tf.data.AUTOTUNE)
        
    return dataset 

In [232]:
def build_model(input_shape, output_shape): 
    inputs = l.Input(input_shape, name='input')
    
    x = l.Dropout(0.3)(inputs)
    
    x = l.Dense(128, activation = 'linear', name='hidden')(x)
    x = l.Dense(64, activation = 'linear', name='hidden1')(x)
#     x = l.BatchNormalization()(inputs)

    outputs = l.Dense(output_shape, activation = 'softmax', name='output')(x)
    
    return Model(inputs, outputs)

In [233]:
train_dataset = build_dataset(X_train, y_train, batch_size = 8)
val_dataset = build_dataset(X_val, y_val, batch_size = 16)

In [234]:
# call the model
model = build_model(X_train.shape[-1], 3)
model.compile(optimizer = Adam(1e-4), loss = SparseCategoricalCrossentropy(), metrics=['accuracy'])

es = tf.keras.callbacks.EarlyStopping(patience = 60, min_delta = 1e-5, restore_best_weights = True)

In [235]:
model.fit(train_dataset, epochs = 200, validation_data = val_dataset, callbacks = [es])

Epoch 1/200
791/791 [==============================] - 2s 2ms/step - loss: 75.6140 - accuracy: 0.5530 - val_loss: 28.8357 - val_accuracy: 0.3669
Epoch 2/200
791/791 [==============================] - 1s 2ms/step - loss: 41.0524 - accuracy: 0.5639 - val_loss: 16.8353 - val_accuracy: 0.6110
Epoch 3/200
791/791 [==============================] - 1s 2ms/step - loss: 31.2061 - accuracy: 0.5803 - val_loss: 23.3225 - val_accuracy: 0.6648
Epoch 4/200
791/791 [==============================] - 1s 2ms/step - loss: 36.8717 - accuracy: 0.5769 - val_loss: 41.8019 - val_accuracy: 0.3435
Epoch 5/200
791/791 [==============================] - 1s 2ms/step - loss: 38.9043 - accuracy: 0.5748 - val_loss: 28.9863 - val_accuracy: 0.6490
Epoch 6/200
791/791 [==============================] - 1s 2ms/step - loss: 29.9340 - accuracy: 0.5829 - val_loss: 51.4904 - val_accuracy: 0.6243
Epoch 7/200
791/791 [==============================] - 1s 2ms/step - loss: 33.9356 - accuracy: 0.5954 - val_loss: 47.2614 - val_ac

In [240]:
test_df = pd.read_csv(test_path)
test_X = preprocess_x(test_df)

In [241]:
prediction = model.predict(test_X)
c_status = prediction[:, 1]
cl_status = prediction[:, 2]
d_status = prediction[:, 0]

df_data = {
    'Status_C': c_status, 
    'Status_CL': cl_status,
    'Status_D': d_status, 
}

165/165 [==============================] - 0s 1ms/step


In [242]:
submission = pd.DataFrame({'id': test_df['id'], **df_data})

In [243]:
submission.to_csv('submission.csv', index=False)